In [ ]:
pip install geopy

In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
import re
from collections import Counter
import geopandas as gpd

In [ ]:
test = pd.read_csv('DIM_TIENDA_TEST.csv')
test

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
0,680,1,C,Hogar,102.36,13,0,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST
1,730,1,C,Hogar,97.43,14,0,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST
2,650,1,C,Hogar,117.01,13,0,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST
3,670,1,C,Base,109.76,13,0,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST
4,800,1,C,Peatonal,0.00,0,0,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,0.00,0,0,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST
101,130,6,CD,Base,110.06,10,0,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
102,320,6,C,Hogar,122.00,10,0,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
103,510,6,B,Hogar,0.00,13,0,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST


In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] == 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test['MTS2VENTAS_NUM'] == 0)
)

# Mostrar las filas donde se cumple la condición
test[condicion].shape[0]

21

In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] == 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test['MTS2VENTAS_NUM'] > 0)
)

# Mostrar las filas donde se cumple la condición
test[condicion].shape[0]

0

In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] == 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] == 0)
)

# Mostrar las filas donde se cumple la condición
test[condicion].shape[0]

0

In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test['MTS2VENTAS_NUM'] == 0)
)

# Mostrar las filas donde se cumple la condición
test[condicion].shape[0]

9

In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] == 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] > 0)
)

# Mostrar las filas donde se cumple la condición
test[condicion].shape[0]

1

In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test['MTS2VENTAS_NUM'] > 0)
)

# Mostrar las filas donde se cumple la condición
test[condicion].shape[0]

33

In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] == 0)
)

# Mostrar las filas donde se cumple la condición
test[condicion].shape[0]

0

In [ ]:
condicion = (
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] > 0)
)

test[condicion].shape[0]

41

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Casos válidos para entrenar el modelo
train_data = test[
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] > 0)
]

In [ ]:
X_train = test[['CAJONESESTACIONAMIENTO_NUM', 'MTS2VENTAS_NUM']]
y_train = test['PUERTASREFRIG_NUM']

modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [ ]:
# Casos donde MTS2VENTAS_NUM está en cero pero los otros no
casos_a_imputar = test[
    (test['PUERTASREFRIG_NUM'] == 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] > 0)
]

X_imputar = casos_a_imputar[['CAJONESESTACIONAMIENTO_NUM', 'MTS2VENTAS_NUM']]
X_imputar

,CAJONESESTACIONAMIENTO_NUM,MTS2VENTAS_NUM
27,13,121.3


In [ ]:
pr = modelo.predict(X_imputar)

In [ ]:
pr.min()

np.float64(12.196721278669528)

In [ ]:
# Copiamos el DataFrame para no modificar el original directamente
test_imputado = test.copy()

# Índices de las filas que queremos imputar
indices_imputar = casos_a_imputar.index

# Imputamos los valores predichos
test_imputado.loc[indices_imputar, 'PUERTASREFRIG_NUM'] = pr

<ipython-input-387-dbec6c5eed96>:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[12.19672128]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_imputado.loc[indices_imputar, 'PUERTASREFRIG_NUM'] = pr


In [ ]:
test_imputado

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
0,680,1,C,Hogar,102.36,13.0,0,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST
1,730,1,C,Hogar,97.43,14.0,0,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST
2,650,1,C,Hogar,117.01,13.0,0,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST
3,670,1,C,Base,109.76,13.0,0,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST
4,800,1,C,Peatonal,0.00,0.0,0,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,0.00,0.0,0,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST
101,130,6,CD,Base,110.06,10.0,0,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
102,320,6,C,Hogar,122.00,10.0,0,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
103,510,6,B,Hogar,0.00,13.0,0,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST


In [ ]:
condicion = (
    (test_imputado['PUERTASREFRIG_NUM'] == 0) &
    (test_imputado['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test_imputado['MTS2VENTAS_NUM'] > 0)
)

# Mostrar las filas donde se cumple la condición
test_imputado[condicion].shape[0]

0

In [ ]:
train_data = test[
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] > 0)
]
X_train = train_data[['PUERTASREFRIG_NUM']]
y_train = train_data['MTS2VENTAS_NUM']

modelo = LinearRegression()
modelo.fit(X_train, y_train)
# Casos donde MTS2VENTAS_NUM está en cero pero los otros no


LinearRegression()

In [ ]:
casos_a_imputar = test[
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test['MTS2VENTAS_NUM'] == 0)
]

X_imputar = casos_a_imputar[['PUERTASREFRIG_NUM']]
X_imputar

,PUERTASREFRIG_NUM
38,8
40,13
48,10
49,13
58,13
66,13
70,13
103,13
104,13


In [ ]:
mts2_predichos = modelo.predict(X_imputar)
# Copiamos el DataFrame para no modificar el original directamente
test_imputado2 = test_imputado.copy()

# Índices de las filas que queremos imputar
indices_imputar = casos_a_imputar.index

# Imputamos los valores predichos
test_imputado2.loc[indices_imputar, 'MTS2VENTAS_NUM'] = mts2_predichos

In [ ]:
test_imputado2

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
0,680,1,C,Hogar,102.360000,13.0,0,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST
1,730,1,C,Hogar,97.430000,14.0,0,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST
2,650,1,C,Hogar,117.010000,13.0,0,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST
3,670,1,C,Base,109.760000,13.0,0,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST
4,800,1,C,Peatonal,0.000000,0.0,0,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,0.000000,0.0,0,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST
101,130,6,CD,Base,110.060000,10.0,0,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
102,320,6,C,Hogar,122.000000,10.0,0,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
103,510,6,B,Hogar,110.952073,13.0,0,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST


In [ ]:
condicion = (
    (test_imputado2['PUERTASREFRIG_NUM'] == 0) &
    (test_imputado2['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test_imputado2['MTS2VENTAS_NUM'] > 0)
)

# Mostrar las filas donde se cumple la condición
test_imputado2[condicion].shape[0]

0

In [ ]:
train_data = test[
    (test['PUERTASREFRIG_NUM'] > 0) &
    (test['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test['MTS2VENTAS_NUM'] > 0)
]
X_train = train_data[['PUERTASREFRIG_NUM', 'MTS2VENTAS_NUM']]
y_train = train_data['CAJONESESTACIONAMIENTO_NUM']


modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [ ]:
casos_a_imputar = test_imputado2[
    (test_imputado2['PUERTASREFRIG_NUM'] > 0) &
    (test_imputado2['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test_imputado2['MTS2VENTAS_NUM'] > 0)
]

In [ ]:
casos_a_imputar.shape

(42, 12)

In [ ]:
X_imputar = casos_a_imputar[['PUERTASREFRIG_NUM', 'MTS2VENTAS_NUM']]
X_imputar

,PUERTASREFRIG_NUM,MTS2VENTAS_NUM
0,13.0,102.360000
1,14.0,97.430000
2,13.0,117.010000
3,13.0,109.760000
6,13.0,128.970000
9,8.0,66.930000
14,12.0,109.860000
22,7.0,83.470000
28,10.0,101.700000
29,10.0,120.000000


In [ ]:
mts2_predichos = modelo.predict(X_imputar)

In [ ]:
test_imputado3 = test_imputado2.copy()

# Índices de las filas que queremos imputar
indices_imputar = casos_a_imputar.index

# Imputamos los valores predichos
test_imputado3.loc[indices_imputar, 'CAJONESESTACIONAMIENTO_NUM'] = mts2_predichos

<ipython-input-400-31ce76566714>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 7.86893791  7.1938308   8.70544734  8.2914751   9.38835878  7.81392838
  8.69079106  9.15196208  9.01207015 10.05699319  9.66870229  8.52748614
  8.35954198  8.89558693 10.19403227  8.60209703  7.52710295  9.08629966
  9.14503817  8.35954198  8.08153812  8.71172616  9.41290948  8.35954198
  9.36989729  8.40548472  8.35954198  9.5516637   8.35954198  8.43022717
  8.29946904  9.12284555  8.16642693  7.92622929  8.27777119  8.9150008
  8.61732003 10.17613963  9.48942297 10.17119243  8.35954198  8.35954198]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_imputado3.loc[indices_imputar, 'CAJONESESTACIONAMIENTO_NUM'] = mts2_predichos


In [ ]:
test_imputado3

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
0,680,1,C,Hogar,102.360000,13.0,7.868938,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST
1,730,1,C,Hogar,97.430000,14.0,7.193831,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST
2,650,1,C,Hogar,117.010000,13.0,8.705447,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST
3,670,1,C,Base,109.760000,13.0,8.291475,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST
4,800,1,C,Peatonal,0.000000,0.0,0.000000,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,0.000000,0.0,0.000000,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST
101,130,6,CD,Base,110.060000,10.0,9.489423,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
102,320,6,C,Hogar,122.000000,10.0,10.171192,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
103,510,6,B,Hogar,110.952073,13.0,8.359542,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST


In [ ]:
condicion = (
    (test_imputado3['PUERTASREFRIG_NUM'] > 0) &
    (test_imputado3['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test_imputado3['MTS2VENTAS_NUM'] > 0)
)

# Mostrar las filas donde se cumple la condición
test_imputado3[condicion].shape[0]

0

In [ ]:
filtro = (
    (test_imputado3['PUERTASREFRIG_NUM'] == 0) &
    (test_imputado3['MTS2VENTAS_NUM'] == 0) &
    (test_imputado3['CAJONESESTACIONAMIENTO_NUM'] == 0)
)

todos0 = test_imputado3[filtro]
todos0

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
4,800,1,C,Peatonal,0.0,0.0,0.0,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
10,790,1,C,Hogar,0.0,0.0,0.0,25.73393,-100.16229,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
17,840,2,C,Hogar,0.0,0.0,0.0,26.02177,-98.26152,Hogar Reunión,UT_DENSIDAD,TEST
18,810,2,B,Base,0.0,0.0,0.0,26.06862,-98.31477,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
24,1040,2,C,Hogar,0.0,0.0,0.0,26.22708,-98.59373,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
25,1050,2,C,Hogar,0.0,0.0,0.0,26.00053,-98.12136,Hogar Reunión,UT_DENSIDAD,TEST
37,10,2,C,Base,0.0,0.0,0.0,26.02203,-98.27759,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
44,630,3,C,Base,0.0,0.0,0.0,25.66690,-100.36854,Oficinistas,UT_TRAFICO_VEHICULAR,TEST
55,710,4,C,Hogar,0.0,0.0,0.0,25.61985,-100.08667,Hogar Reunión,UT_DENSIDAD,TEST
56,660,4,B,Base,0.0,0.0,0.0,25.64418,-100.08754,Oficinistas,UT_TRAFICO_VEHICULAR,TEST


In [ ]:
# Copia del DataFrame original
test_montecarlo = test_imputado3.copy()

# Valores válidos para ajustar la simulación
valores_validos_mts2 = test_montecarlo[test_montecarlo['MTS2VENTAS_NUM'] > 0]['MTS2VENTAS_NUM']
valores_validos_refri = test_montecarlo[test_montecarlo['PUERTASREFRIG_NUM'] > 0]['PUERTASREFRIG_NUM']

# Ajuste de distribución para MTS2VENTAS_NUM
mu_mts2 = valores_validos_mts2.mean()
sigma_mts2 = valores_validos_mts2.std()

# Filtro: solo donde AMBAS variables son cero
filtro_ceros = (
    (test_montecarlo['MTS2VENTAS_NUM'] == 0) &
    (test_montecarlo['PUERTASREFRIG_NUM'] == 0)
)

n = filtro_ceros.sum()

# Imputación Montecarlo
valores_simulados_mts2 = np.random.normal(mu_mts2, sigma_mts2, n)
valores_simulados_refri = np.random.choice(valores_validos_refri, size=n, replace=True)

# Asignar valores simulados
test_montecarlo.loc[filtro_ceros, 'MTS2VENTAS_NUM'] = valores_simulados_mts2
test_montecarlo.loc[filtro_ceros, 'PUERTASREFRIG_NUM'] = valores_simulados_refri

In [ ]:
test_montecarlo[filtro]

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
4,800,1,C,Peatonal,120.480242,14.0,0.0,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
10,790,1,C,Hogar,99.892523,12.0,0.0,25.73393,-100.16229,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
17,840,2,C,Hogar,118.441854,10.0,0.0,26.02177,-98.26152,Hogar Reunión,UT_DENSIDAD,TEST
18,810,2,B,Base,93.920734,15.0,0.0,26.06862,-98.31477,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
24,1040,2,C,Hogar,67.106958,13.0,0.0,26.22708,-98.59373,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
25,1050,2,C,Hogar,87.714454,10.0,0.0,26.00053,-98.12136,Hogar Reunión,UT_DENSIDAD,TEST
37,10,2,C,Base,93.408466,13.0,0.0,26.02203,-98.27759,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
44,630,3,C,Base,100.242867,13.0,0.0,25.66690,-100.36854,Oficinistas,UT_TRAFICO_VEHICULAR,TEST
55,710,4,C,Hogar,89.653034,13.0,0.0,25.61985,-100.08667,Hogar Reunión,UT_DENSIDAD,TEST
56,660,4,B,Base,95.028219,13.0,0.0,25.64418,-100.08754,Oficinistas,UT_TRAFICO_VEHICULAR,TEST


In [ ]:
# Casos válidos para entrenar el modelo
train_data = test_montecarlo[
    (test_montecarlo['PUERTASREFRIG_NUM'] > 0) &
    (test_montecarlo['CAJONESESTACIONAMIENTO_NUM'] > 0) &
    (test_montecarlo['MTS2VENTAS_NUM'] > 0)
]
X_train = train_data[['PUERTASREFRIG_NUM', 'MTS2VENTAS_NUM']]
y_train = train_data['CAJONESESTACIONAMIENTO_NUM']

modelo = LinearRegression()
modelo.fit(X_train, y_train)
# Casos donde MTS2VENTAS_NUM está en cero pero los otros no

LinearRegression()

In [ ]:
casos_a_imputar  = test_montecarlo[
    (test_montecarlo['PUERTASREFRIG_NUM'] > 0) &
    (test_montecarlo['CAJONESESTACIONAMIENTO_NUM'] == 0) &
    (test_montecarlo['MTS2VENTAS_NUM'] > 0)
]

X_imputar = casos_a_imputar[['PUERTASREFRIG_NUM', 'MTS2VENTAS_NUM']]
X_imputar

,PUERTASREFRIG_NUM,MTS2VENTAS_NUM
4,14.0,120.480242
10,12.0,99.892523
17,10.0,118.441854
18,15.0,93.920734
24,13.0,67.106958
25,10.0,87.714454
37,13.0,93.408466
44,13.0,100.242867
55,13.0,89.653034
56,13.0,95.028219


In [ ]:
mts2_predichos = modelo.predict(X_imputar)
# Copiamos el DataFrame para no modificar el original directamente
test_limpio = test_montecarlo.copy()

# Índices de las filas que queremos imputar
indices_imputar = casos_a_imputar.index

# Imputamos los valores predichos
test_limpio.loc[indices_imputar, 'CAJONESESTACIONAMIENTO_NUM'] = mts2_predichos

In [ ]:
a = test_limpio[filtro]

In [ ]:
test_limpio

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET
0,680,1,C,Hogar,102.360000,13.0,7.868938,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST
1,730,1,C,Hogar,97.430000,14.0,7.193831,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST
2,650,1,C,Hogar,117.010000,13.0,8.705447,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST
3,670,1,C,Base,109.760000,13.0,8.291475,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST
4,800,1,C,Peatonal,120.480242,14.0,8.587192,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,100.258370,10.0,8.952197,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST
101,130,6,CD,Base,110.060000,10.0,9.489423,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
102,320,6,C,Hogar,122.000000,10.0,10.171192,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST
103,510,6,B,Hogar,110.952073,13.0,8.359542,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST


In [ ]:
denueNL=pd.read_csv("denue_inegi_19_.csv",encoding="latin1")
denueNL

,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,9821211,19009115111000021000000000U3,AGR SERKA AGROQUIMICOS Y SERVICIOS CADEREYTA,NaN,115111,Servicios de fumigación agrícola,0 a 5 personas,CALLE,ABASOLO,CALLE,...,Cadereyta Jiménez ...,0481,1,8.281087e+09,NaN,NaN,Fijo,25.589012,-100.009229,2024-11
1,8947881,19039431150000271000000000U6,AGROPECUARIA DE GRANOS E INSUMOS,AGROPECUARIA DE GRANOS E INSUMOS SA DE CV,115119,Otros servicios relacionados con la agricultura,0 a 5 personas,AVENIDA,LAZARO CARDENAS,CALLE,...,Monterrey ...,2307,11,NaN,NaN,NaN,Fijo,25.617114,-100.278360,2019-11
2,11507742,19018115119000022000000000S5,ARPSA PETROLAP,ARPSA PETROLAP SA DE CV,115119,Otros servicios relacionados con la agricultura,6 a 10 personas,CALLE,DELTA,CALLE,...,Parque Industrial Ciudad Mitras ...,0160,12,8.119650e+09,CALIDAD@ARSAPETROLAP.COM,WWW.ARSAPETROLAP.COM,Fijo,25.791256,-100.430177,2024-11
3,10920746,19026115210000021000000000U7,BAÑOS PUBLICOS MERCADO,NaN,115210,Servicios relacionados con la cría y explotaci...,0 a 5 personas,CALLE,INDEPENDENCIA,CALLE,...,Guadalupe ...,0317,11,8.126273e+09,NaN,NaN,Fijo,25.678788,-100.252304,2024-11
4,10171494,19044115210000021000000000U0,BASCULA PUBLICA,NaN,115210,Servicios relacionados con la cría y explotaci...,0 a 5 personas,CALLE,NACIONAL,CALLE,...,Ciudad Sabinas Hidalgo ...,0461,6,8.241063e+09,NaN,NaN,Fijo,26.520831,-100.166979,2024-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207776,10244581,19019931210000661000000000U7,W TOP,WTOP SA DE CV,931210,Administración pública en general,0 a 5 personas,CALLE,16 DE SEPTIEMBRE,CALLE,...,San Pedro Garza García ...,0581,12,NaN,NaN,NaN,Fijo,25.661329,-100.407925,2024-11
207777,10129026,19039931610002481000000000U7,ZONA 2 EDUCACION ESPECIAL EDUCACION BASICA,INSPECCION EDUCACION ESPECIALALUMNOS EDUCACION...,931610,Actividades administrativas de instituciones d...,0 a 5 personas,CALLE,NANTES,CALLE,...,Monterrey ...,2025,5,NaN,ZONAII.EDESP@GMAIL.COM,NaN,Fijo,25.650646,-100.282881,2024-11
207778,10640592,19014611699000121000000000U4,ZONA ESCOLAR 09 TELESECUDARIA,NaN,931610,Actividades administrativas de instituciones d...,0 a 5 personas,CALLE,IGNACIO LOPEZ RAYON,CALLE,...,Doctor Arroyo ...,0551,35,NaN,NaN,NaN,Fijo,23.666786,-100.181830,2024-11
207779,10461159,19039931610002501000000000U2,ZONA III,ZONA III,931610,Actividades administrativas de instituciones d...,0 a 5 personas,CALLE,PLAN DE GUADALUPE,CALLE,...,Monterrey ...,0870,23,NaN,NaN,NaN,Fijo,25.704004,-100.356737,2024-11


In [ ]:
# Lista de actividades económicas de interés
actividades_interes = [
    "Comercio al por menor en tiendas de abarrotes, ultramarinos y misceláneas",
    "Comercio al por menor de artículos usados",
    "Comercio al por menor en supermercados",
    "Comercio al por menor en minisupers",
    "Cafeterías, fuentes de sodas, neverías, refresquerías y similares"
]

# Filtrar el DataFrame del DENUE
df_denue_filtrado = denueNL[denueNL['nombre_act'].isin(actividades_interes)].copy()

In [ ]:
df_denue_filtrado

,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
26894,11169774,19031461110027581000000000U1,2 HERMANAS,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,CALLE,ESTELLA,CALLE,...,Ciudad Benito Juárez ...,2084,13,8.281192e+09,NaN,NaN,Fijo,25.673254,-100.080229,2024-11
26940,11286868,19001461110000501000000000U3,4 DE AGOSTO,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,CALLE,ENRIQUE VILLARREAL GUTIERREZ,CALLE,...,Alberto Villarreal ...,0017,31,NaN,NaN,NaN,Fijo,25.939706,-100.404447,2024-11
26949,9330966,19019462111000234000003517S0,419 CITY MARKET GOMEZ MORIN,COMERCIAL CITY FRESKO S DE RL DE CV,462111,Comercio al por menor en supermercados,0 a 5 personas,AVENIDA,MANUEL GOMEZ MORIN,CALLE,...,San Pedro Garza García ...,0384,1,NaN,YPALACIOS@LACOMER.COM.MX,WWW.LACOMER.COM.MX,Fijo,25.651301,-100.358299,2020-11
26955,9411517,19039462111001766000003517S8,438 FRESKO CUMBRES,COMERCIAL CITY FRESKO S DE RL DE CV,462111,Comercio al por menor en supermercados,0 a 5 personas,AVENIDA,PASEO DE LOS LEONES,AVENIDA,...,Monterrey ...,4801,1,NaN,YPALACIOS@LACOMER.COM.MX,LACOMER.COM.MX,Fijo,25.732935,-100.400132,2023-11
26956,9398599,19039462111001686000003517S8,439 FRESKO ESFERA,COMERCIAL CITY FRESKO S DE RL DE CV,462111,Comercio al por menor en supermercados,0 a 5 personas,AVENIDA,LA RIOJA,RETORNO,...,Monterrey ...,2985,53,NaN,YPALACIOS@LACOMER.COM.MX,LACOMER.COM.MX,Fijo,25.574771,-100.243776,2023-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169032,9534672,19021722515002581000084770S6,YUKAPIOCA,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,CALLE,NINGUNO,PROLONGACION,...,Ciudad General Escobedo ...,0325,4,7.221688e+09,FRANQUICIAS@YUKAPIOCA.COM,NaN,Semifijo,25.766703,-100.303417,2024-11
169065,11756887,19026722515005511000000000U9,ZOMBIE PAPAS,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,CALLE,Vista Regia,AVENIDA,...,Guadalupe ...,0162,13,NaN,NaN,NaN,Fijo,25.692876,-100.254177,2025-04
169072,8751434,19019722515000701000000000U6,ZU CAFE,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,CALLE,RIO AMAZONAS,CALLE,...,San Pedro Garza García ...,0083,24,NaN,ZUCAFEMTY@HOTMAIL.COM,NaN,Fijo,25.659477,-100.361581,2019-11
169073,2973815,19039722515002281000000000U5,ZU CAFE,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,CALLE,WASHINGTON,AVENIDA,...,Monterrey ...,1690,38,8.183406e+09,NaN,NaN,Fijo,25.674363,-100.314161,2014-12


In [ ]:
R = 6371000  # Radio de la Tierra en metros
distancia_maxima = 200  # Puedes ajustar este valor (en metros)

In [ ]:
def haversine_np(lat1, lon1, lat2, lon2):
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

In [ ]:
def categorizar(nombre_act, nombre_comercial):
    nombre_act = str(nombre_act).lower()
    nombre_comercial = str(nombre_comercial).lower()

    if re.search(r'\boxxo\b', nombre_comercial):
        return "OXXO"
    elif re.search(r'seven|7\s?eleven', nombre_comercial):
        return "Seven-Eleven"
    elif re.search(r'farmacia|farmacias|similares', nombre_comercial):
        return "Farmacias"
    elif re.search(r'supermercado|super\b|autoservicio', nombre_act):
        return "Supers"
    elif re.search(r'abarrote|miscelánea|ultramarino', nombre_act):
        return "Abarrotes"
    elif re.search(r'cafetería|nevería|refresquería|fuente de sodas', nombre_act):
        return "Cafeterías"
    elif re.search(r'ropa usada|artículo usado|segunda mano', nombre_act):
        return "Usados"
    else:
        return "Otros"

In [ ]:
df_denue_filtrado["categoria"] = df_denue_filtrado.apply(
    lambda row: categorizar(row["nombre_act"], row.get("nom_estab", "")), axis=1
)

In [ ]:
resultados = []

for idx, tienda in test.iterrows():
    lat_oxxo = tienda["LATITUD_NUM"]
    lon_oxxo = tienda["LONGITUD_NUM"]
    tienda_id = tienda["TIENDA_ID"]

    # Calcular distancias desde la tienda actual a todos los negocios DENUE
    distancias = haversine_np(lat_oxxo, lon_oxxo,
                              df_denue_filtrado["latitud"], df_denue_filtrado["longitud"])

    # Filtrar negocios cercanos
    negocios_cercanos = df_denue_filtrado[distancias <= distancia_maxima]

    # Contar por categoría
    conteo = Counter(negocios_cercanos["categoria"])

    # Guardar resultado para esta tienda
    resultado = {"TIENDA_ID": tienda_id}
    resultado.update(conteo)
    resultados.append(resultado)

In [ ]:
df_resultado = pd.DataFrame(resultados).fillna(0)

In [ ]:
categorias = ["OXXO", "Seven-Eleven", "Supers", "Farmacias", "Abarrotes", "Cafeterías", "Usados", "Otros"]
for cat in categorias:
    if cat not in df_resultado.columns:
        df_resultado[cat] = 0

In [ ]:
columns_to_check = ['OXXO', 'Seven-Eleven', 'Supers', 'Farmacias',
                    'Abarrotes', 'Cafeterías', 'Usados', 'Otros']

# Drop rows where all specified columns are 0
data = df_resultado[~(df_resultado[columns_to_check] == 0).all(axis=1)]
data

,TIENDA_ID,Seven-Eleven,Abarrotes,Otros,OXXO,Cafeterías,Supers,Farmacias,Usados
0,680,1.0,5.0,1.0,2.0,0.0,0.0,0,0
1,730,0.0,1.0,0.0,1.0,1.0,0.0,0,0
2,650,0.0,2.0,0.0,1.0,1.0,0.0,0,0
3,670,0.0,1.0,0.0,1.0,0.0,0.0,0,0
4,800,0.0,1.0,0.0,0.0,2.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...
100,720,0.0,5.0,0.0,2.0,0.0,0.0,0,0
101,130,0.0,2.0,0.0,1.0,0.0,0.0,0,0
102,320,0.0,3.0,1.0,0.0,1.0,0.0,0,0
103,510,0.0,3.0,2.0,1.0,0.0,0.0,0,0


##### tamps


In [ ]:
denue_tamps=pd.read_csv("denue_inegi_28_.csv",encoding="latin1")
denue_tamps

,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,8203497,28002112512000023000000000U5,7 ADELANTE AGRICULTORES,7 ADELANTE AGRICULTORES S DE SS,112513,Camaronicultura y acuicultura de otros crustác...,0 a 5 personas,CALLE,Sebastián Lerdo de Tejada,CALLE,...,Aldama ...,0456,3,NaN,NaN,NaN,Fijo,22.915794,-98.077246,2019-11
1,8011852,28002114119000503000000000U8,ACUACAM,ACUACAM SA DE CV,112513,Camaronicultura y acuicultura de otros crustác...,0 a 5 personas,CALLE,CAMINO AL EJIDO PRESITAS,OTRO(ESPECIFIQUE),...,Aldama ...,0441,7,NaN,NaN,NaN,Fijo,22.923666,-98.075615,2019-11
2,6182392,28002112511000063000000000U3,ACUACULTIVO EL TORAL,ACUACULTIVO EL TORAL SA DE CV,112513,Camaronicultura y acuicultura de otros crustác...,0 a 5 personas,CALLE,NINGUNO,OTRO(ESPECIFIQUE),...,Morón ...,0653,1,NaN,NaN,NaN,Fijo,22.732457,-97.865414,2011-03
3,8178768,28001112512000062000000000O5,ACUACULTIVOS EL PORVENIR,ACUACULTIVOS EL PORVENIR SA DE CV,112515,Piscicultura en agua dulce,0 a 5 personas,CALLE,Ninguno,CALLE,...,Abasolo (Abasolito) ...,0406,1,NaN,NaN,NaN,Fijo,24.043485,-98.383674,2019-11
4,8789487,28001112512000041000000000O9,ACUACULTIVOS FAZ MORA,ACUACULTIVOS FAZ MORA S DE RL MI,112515,Piscicultura en agua dulce,0 a 5 personas,OTRO(ESPECIFIQUE),RIO SOTO LA MARINA,CALLE,...,Guía del Porvenir ...,0406,800,8.342660e+09,NaN,NaN,Fijo,23.996400,-98.428886,2019-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148555,7984712,28021611112000371000000000U3,ZONA 30 SUPERVISION FEDERAL DE EDUCACION PREES...,SECRETARIA DE EDUCACION PUBLICA,931610,Actividades administrativas de instituciones d...,0 a 5 personas,CALLE,ARGENTINA,CALLE,...,Ciudad Mante ...,0502,51,8.311371e+09,NaN,NaN,Fijo,22.736384,-98.955534,2019-11
148556,4091799,28038931510000067001000000U9,ZONA DE TRANSFERENCIAS DEL MUNICIPIO DE TAMPICO,MUNICIPIO DE TAMPICO TAM,931510,Regulación y fomento de actividades para mejor...,101 a 250 personas,CARRETERA,FEDERAL NUMERO 70,CARRETERA,...,Tampico ...,1404,31,8.333053e+09,NaN,NaN,Fijo,22.225992,-97.886972,2014-12
148557,4118890,28041931610000161000000000U2,ZONA ESCOLAR 150 DE GUEMEZ,SECRETARIA DE EDUCACION PUBLICA,931610,Actividades administrativas de instituciones d...,6 a 10 personas,CALLE,CANELO,CALLE,...,Ciudad Victoria ...,1005,9,NaN,28FIZ0150G@SET.EDU.MX,NaN,Fijo,23.728478,-99.168491,2010-07
148558,4063247,28040931610000201000000000U0,ZONA ESCOLAR 38,SECRETARIA DE EDUCACION PUBLICA,931610,Actividades administrativas de instituciones d...,0 a 5 personas,CALLE,AMERICA,CALLE,...,Valle Hermoso ...,0461,1,NaN,ZONAESCOLAR38@SEP.EDU.MX,NaN,Fijo,25.675619,-97.817680,2014-12


In [ ]:
# Filtrar el DataFrame del DENUE
df_denue_filtrado = denue_tamps[denue_tamps['nombre_act'].isin(actividades_interes)].copy()

In [ ]:
df_denue_filtrado

,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
15059,4059250,28033466410004761000000000U0,ARTICULOS VARIOS SIN NOMBRE,NaN,466410,Comercio al por menor de artículos usados,0 a 5 personas,OTRO(ESPECIFIQUE),NINGUNO,CALLE,...,Ciudad Río Bravo ...,0486,38,NaN,NaN,NaN,Semifijo,25.993604,-98.120144,2014-12
15099,4106788,28003461110007651000000000U4,2 DE MAYO,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,CALLE,GENERAL LAZARO CARDENAS DEL RIO,CALLE,...,Miramar ...,0612,52,NaN,NaN,NaN,Fijo,22.334953,-97.868508,2010-07
15109,4088956,28003466410000591000000000U8,3 ABRILES,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,CALLE,RIO AMAZONAS,CALLE,...,Miramar ...,0862,33,NaN,NaN,NaN,Fijo,22.354003,-97.853341,2014-12
15112,4045369,28021461110007911000000000U4,3 HERMANOS,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,CALLE,BELGICA,CALLE,...,Ciudad Mante ...,152A,4,NaN,NaN,NaN,Fijo,22.704428,-98.978087,2013-07
15113,4072710,28027461110024311000000000U3,3 HERMANOS,NaN,461110,"Comercio al por menor en tiendas de abarrotes,...",0 a 5 personas,CALLE,BRASILIA,CALLE,...,El Campanario y Oradel ...,2860,18,8.671565e+09,NaN,NaN,Fijo,27.468763,-99.616189,2014-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115626,11513551,28038722515002741000000000S9,ZUMBA JUGOS & SMOTHIES,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,AVENIDA,AYUNTAMIENTO,CALLE,...,Tampico ...,0209,33,NaN,ZUMBASMOOTHIESYJUGOS@HOTMAIL.COM,NaN,Fijo,22.240555,-97.866783,2024-11
115627,10070873,28009722515001361000000000U8,ZUMBA JUGOS Y LICUADOS,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,AVENIDA,FRANCISCO I MADERO,CALLE,...,Ciudad Madero ...,0777,8,NaN,NaN,NaN,Fijo,22.252741,-97.831153,2024-11
115628,8821345,28009722515000621000000000U6,ZUMBA JUGOS Y SMOOTHIES,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,BOULEVARD,ADOLFO LOPEZ MATEOS,CALLE,...,Ciudad Madero ...,0546,15,NaN,NaN,NaN,Fijo,22.272244,-97.850388,2019-11
115629,4040210,28038722219005602000000000M9,ZUMBA JUGOS Y SMOOTHIS,NaN,722515,"Cafeterías, fuentes de sodas, neverías, refres...",0 a 5 personas,CALLE,LOMA BONITA,CALLE,...,Tampico ...,0073,16,8.332135e+09,ZUMBASMOOTHESYJUGOS@HOTMAIL.COM,NaN,Fijo,22.265836,-97.872776,2010-07


In [ ]:
df_denue_filtrado["categoria"] = df_denue_filtrado.apply(
    lambda row: categorizar(row["nombre_act"], row.get("nom_estab", "")), axis=1
)

In [ ]:
resultados = []

for idx, tienda in test.iterrows():
    lat_oxxo = tienda["LATITUD_NUM"]
    lon_oxxo = tienda["LONGITUD_NUM"]
    tienda_id = tienda["TIENDA_ID"]

    # Calcular distancias desde la tienda actual a todos los negocios DENUE
    distancias = haversine_np(lat_oxxo, lon_oxxo,
                              df_denue_filtrado["latitud"], df_denue_filtrado["longitud"])

    # Filtrar negocios cercanos
    negocios_cercanos = df_denue_filtrado[distancias <= distancia_maxima]

    # Contar por categoría
    conteo = Counter(negocios_cercanos["categoria"])

    # Guardar resultado para esta tienda
    resultado = {"TIENDA_ID": tienda_id}
    resultado.update(conteo)
    resultados.append(resultado)

In [ ]:
df_resultado = pd.DataFrame(resultados).fillna(0)

In [ ]:
categorias = ["OXXO", "Seven-Eleven", "Supers", "Farmacias", "Abarrotes", "Cafeterías", "Usados", "Otros"]
for cat in categorias:
    if cat not in df_resultado.columns:
        df_resultado[cat] = 0

In [ ]:
columns_to_check = ['OXXO', 'Seven-Eleven', 'Supers', 'Farmacias',
                    'Abarrotes', 'Cafeterías', 'Usados', 'Otros']

# Drop rows where all specified columns are 0
data2 = df_resultado[~(df_resultado[columns_to_check] == 0).all(axis=1)]
data2

,TIENDA_ID,Abarrotes,Otros,Cafeterías,OXXO,Supers,Seven-Eleven,Farmacias,Usados
17,840,5.0,85.0,1.0,0.0,0.0,0.0,0,0
18,810,2.0,1.0,6.0,0.0,0.0,0.0,0,0
19,990,4.0,1.0,0.0,1.0,0.0,0.0,0,0
20,1000,2.0,3.0,1.0,3.0,0.0,0.0,0,0
21,1010,2.0,2.0,0.0,1.0,0.0,0.0,0,0
22,970,1.0,2.0,4.0,2.0,1.0,3.0,0,0
23,980,0.0,0.0,0.0,1.0,0.0,0.0,0,0
24,1040,3.0,1.0,3.0,1.0,0.0,0.0,0,0
25,1050,9.0,4.0,0.0,1.0,0.0,0.0,0,0
26,1020,1.0,0.0,0.0,1.0,0.0,0.0,0,0


In [ ]:
denue_info=pd.concat([data, data2])
denue_info

,TIENDA_ID,Seven-Eleven,Abarrotes,Otros,OXXO,Cafeterías,Supers,Farmacias,Usados
0,680,1.0,5.0,1.0,2.0,0.0,0.0,0,0
1,730,0.0,1.0,0.0,1.0,1.0,0.0,0,0
2,650,0.0,2.0,0.0,1.0,1.0,0.0,0,0
3,670,0.0,1.0,0.0,1.0,0.0,0.0,0,0
4,800,0.0,1.0,0.0,0.0,2.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...
32,950,0.0,2.0,2.0,0.0,0.0,0.0,0,0
33,960,2.0,1.0,0.0,1.0,2.0,0.0,0,0
34,930,0.0,2.0,2.0,1.0,1.0,0.0,0,0
36,940,0.0,4.0,4.0,1.0,0.0,0.0,0,0


In [ ]:
unidos1=test_limpio.merge(denue_info,on="TIENDA_ID",how="left")

In [ ]:
unidos1

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Seven-Eleven,Abarrotes,Otros,OXXO,Cafeterías,Supers,Farmacias,Usados
0,680,1,C,Hogar,102.360000,13.0,7.868938,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST,1.0,5.0,1.0,2.0,0.0,0.0,0.0,0.0
1,730,1,C,Hogar,97.430000,14.0,7.193831,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,650,1,C,Hogar,117.010000,13.0,8.705447,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0
3,670,1,C,Base,109.760000,13.0,8.291475,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,800,1,C,Peatonal,120.480242,14.0,8.587192,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,100.258370,10.0,8.952197,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0
101,130,6,CD,Base,110.060000,10.0,9.489423,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
102,320,6,C,Hogar,122.000000,10.0,10.171192,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0
103,510,6,B,Hogar,110.952073,13.0,8.359542,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,0.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
unidos1.isna().sum()

,0
TIENDA_ID,0
PLAZA_CVE,0
NIVELSOCIOECONOMICO_DES,0
ENTORNO_DES,0
MTS2VENTAS_NUM,0
PUERTASREFRIG_NUM,0
CAJONESESTACIONAMIENTO_NUM,0
LATITUD_NUM,0
LONGITUD_NUM,0
SEGMENTO_MAESTRO_DESC,0


In [ ]:
cols_objetivo = ['OXXO', 'Seven-Eleven', 'Supers', 'Farmacias',
                 'Abarrotes', 'Cafeterías', 'Usados', 'Otros']

unidos1[cols_objetivo] = unidos1[cols_objetivo].fillna(0)

In [ ]:
unidos1.isna().sum()

,0
TIENDA_ID,0
PLAZA_CVE,0
NIVELSOCIOECONOMICO_DES,0
ENTORNO_DES,0
MTS2VENTAS_NUM,0
PUERTASREFRIG_NUM,0
CAJONESESTACIONAMIENTO_NUM,0
LATITUD_NUM,0
LONGITUD_NUM,0
SEGMENTO_MAESTRO_DESC,0


In [ ]:
unidos1

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,DATASET,Seven-Eleven,Abarrotes,Otros,OXXO,Cafeterías,Supers,Farmacias,Usados
0,680,1,C,Hogar,102.360000,13.0,7.868938,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,TEST,1.0,5.0,1.0,2.0,0.0,0.0,0.0,0.0
1,730,1,C,Hogar,97.430000,14.0,7.193831,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,TEST,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
2,650,1,C,Hogar,117.010000,13.0,8.705447,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,TEST,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0
3,670,1,C,Base,109.760000,13.0,8.291475,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,TEST,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,800,1,C,Peatonal,120.480242,14.0,8.587192,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,TEST,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,100.258370,10.0,8.952197,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,TEST,0.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0
101,130,6,CD,Base,110.060000,10.0,9.489423,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
102,320,6,C,Hogar,122.000000,10.0,10.171192,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0
103,510,6,B,Hogar,110.952073,13.0,8.359542,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,TEST,0.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
unidos1.to_csv("df_final_test.csv")

In [ ]:
df_oxxo = pd.read_csv("df_final_test.csv")
df_oxxo

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,LID_UBICACION_TIENDA,Seven-Eleven,Abarrotes,Otros,OXXO,Cafeterías,Supers,Farmacias,Usados,PORCENTAJE_CUMPLIMIENTO
0,680,1,C,Hogar,102.360000,13.0,7.868938,25.65488,-100.21207,Hogar Reunión,UT_DENSIDAD,1,5,1,2,0,0,0,0,1.327246
1,730,1,C,Hogar,97.430000,14.0,7.193831,25.66315,-100.22738,Hogar Reunión,UT_DENSIDAD,0,1,0,1,1,0,0,0,2.736564
2,650,1,C,Hogar,117.010000,13.0,8.705447,25.66404,-100.22993,Hogar Reunión,UT_DENSIDAD,0,2,0,1,1,0,0,0,1.964698
3,670,1,C,Base,109.760000,13.0,8.291475,25.66508,-100.26338,Hogar Reunión,UT_DENSIDAD,0,1,0,1,0,0,0,0,1.646274
4,800,1,C,Peatonal,120.480242,14.0,8.587192,25.69367,-100.21433,Parada Técnica,UT_TRAFICO_PEATONAL,0,1,0,0,2,0,0,0,1.244958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,100.258370,10.0,8.952197,25.60253,-99.99317,Hogar Reunión,UT_DENSIDAD,0,5,0,2,0,0,0,0,1.414713
101,130,6,CD,Base,110.060000,10.0,9.489423,25.36439,-100.10202,Hogar Reunión,UT_TRAFICO_VEHICULAR,0,2,0,1,0,0,0,0,4.021708
102,320,6,C,Hogar,122.000000,10.0,10.171192,25.57214,-100.01371,Hogar Reunión,UT_TRAFICO_VEHICULAR,0,3,1,0,1,0,0,0,2.380020
103,510,6,B,Hogar,110.952073,13.0,8.359542,25.19747,-99.83198,Hogar Reunión,UT_TRAFICO_VEHICULAR,0,3,2,1,0,0,0,0,1.919734


In [ ]:
df_demografico = pd.read_csv('NL demograficas.csv')

In [ ]:
df_demograficoNL2 = df_demografico.copy()
df_demograficoNL2['llaveEnt'] = '19'+df_demograficoNL2['Llave']
df_demograficoNL2

,Llave,P_18A24,PROM_HNV,VIVPAR_UT,PSINDER,VIVTOT,TOTHOG,PDER_SS,GRAPROES,POBTOT,P_60YMAS,P_18YMAS,llaveEnt
0,00000000000,702241,1.88,95820,1072664,2037695,1655256,4681465,10.740000,5784442,654050,4111381,1900000000000
1,00100000000,358,2.17,143,141,1194,802,2818,9.450000,2974,328,2007,1900100000000
2,00100010000,227,2.21,84,14,795,544,1963,9.720000,1992,259,1381,1900100010000
3,00100010021,224,74.75,50,11,884,605,2144,8.290732,2228,278,1523,1900100010021
4,00100010036,183,51.11,60,4,650,416,1564,7.701724,1606,182,1075,1900100010036
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2916,05100010292,0,1.66,0,0,8,6,22,10.890000,22,0,16,1905100010292
2917,05100010305,15,3.60,10,10,38,21,69,4.730000,82,0,56,1905100010305
2918,0510001031A,0,0.00,0,0,8,0,0,0.000000,0,0,0,190510001031A
2919,05100010324,0,3.66,0,0,22,18,60,8.830000,60,12,38,1905100010324


In [ ]:
df_demografico_tms = pd.read_csv('TAM llave.csv')
df_demografico_tms

,Llave,PDER_SS,P_18YMAS,PSINDER,P_60YMAS,POBTOT,VIVTOT,PROM_HNV,GRAPROES,TOTHOG,VIVPAR_UT,P_18A24
0,00000000000,2803407,2475161,713859,426227,3527735,1336750,2.01,10.09,1069121,56233,403631
1,00100000000,9079,6704,734,1762,9822,4573,2.75,7.91,3085,273,786
2,00100010000,4768,3576,483,874,5255,2308,2.56,8.69,1637,154,447
3,00100010177,2150,1660,252,381,2466,1058,112.32,7.047254902,742,15,201
4,00100010213,1035,760,102,138,1188,566,90.93,8.5705,380,47,40
...,...,...,...,...,...,...,...,...,...,...,...,...
2542,04303000270,1179,921,81,176,1278,554,58.59,9.195185185,402,52,127
2543,04303000336,778,618,119,125,936,364,39.41,7.959047619,271,11,83
2544,04303000340,951,736,32,166,1014,442,29.66,6.688461538,323,32,85
2545,04303000355,308,254,28,43,340,126,20.78,6.883571429,100,5,39


In [ ]:
df_demograficoTAM2 = df_demografico_tms.copy()
df_demograficoTAM2['llaveEnt'] = '28'+df_demograficoTAM2['Llave']
df_demograficoTAM2

,Llave,PDER_SS,P_18YMAS,PSINDER,P_60YMAS,POBTOT,VIVTOT,PROM_HNV,GRAPROES,TOTHOG,VIVPAR_UT,P_18A24,llaveEnt
0,00000000000,2803407,2475161,713859,426227,3527735,1336750,2.01,10.09,1069121,56233,403631,2800000000000
1,00100000000,9079,6704,734,1762,9822,4573,2.75,7.91,3085,273,786,2800100000000
2,00100010000,4768,3576,483,874,5255,2308,2.56,8.69,1637,154,447,2800100010000
3,00100010177,2150,1660,252,381,2466,1058,112.32,7.047254902,742,15,201,2800100010177
4,00100010213,1035,760,102,138,1188,566,90.93,8.5705,380,47,40,2800100010213
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,04303000270,1179,921,81,176,1278,554,58.59,9.195185185,402,52,127,2804303000270
2543,04303000336,778,618,119,125,936,364,39.41,7.959047619,271,11,83,2804303000336
2544,04303000340,951,736,32,166,1014,442,29.66,6.688461538,323,32,85,2804303000340
2545,04303000355,308,254,28,43,340,126,20.78,6.883571429,100,5,39,2804303000355


In [ ]:
demograf_all = pd.concat([df_demograficoNL2,df_demograficoTAM2]).reset_index(drop=True)

In [ ]:
demograf_all

,Llave,P_18A24,PROM_HNV,VIVPAR_UT,PSINDER,VIVTOT,TOTHOG,PDER_SS,GRAPROES,POBTOT,P_60YMAS,P_18YMAS,llaveEnt
0,00000000000,702241,1.88,95820,1072664,2037695,1655256,4681465,10.74,5784442,654050,4111381,1900000000000
1,00100000000,358,2.17,143,141,1194,802,2818,9.45,2974,328,2007,1900100000000
2,00100010000,227,2.21,84,14,795,544,1963,9.72,1992,259,1381,1900100010000
3,00100010021,224,74.75,50,11,884,605,2144,8.290732,2228,278,1523,1900100010021
4,00100010036,183,51.11,60,4,650,416,1564,7.701724,1606,182,1075,1900100010036
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5463,04303000270,127,58.59,52,81,554,402,1179,9.195185185,1278,176,921,2804303000270
5464,04303000336,83,39.41,11,119,364,271,778,7.959047619,936,125,618,2804303000336
5465,04303000340,85,29.66,32,32,442,323,951,6.688461538,1014,166,736,2804303000340
5466,04303000355,39,20.78,5,28,126,100,308,6.883571429,340,43,254,2804303000355


In [ ]:
ageb = gpd.read_file('19a.shp')
ageb

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry
0,1900100010021,19,001,0001,0021,"POLYGON ((2659616.6 1545244.086, 2659704.062 1..."
1,1900100010036,19,001,0001,0036,"POLYGON ((2660141.835 1544653.818, 2660161.176..."
2,1900100010055,19,001,0001,0055,"POLYGON ((2660414.5 1543978.139, 2660414.199 1..."
3,1900100010040,19,001,0001,0040,"POLYGON ((2660300.222 1545075.583, 2661109.051..."
4,1900200010230,19,002,0001,0230,"POLYGON ((2744964.21 1586383.575, 2744955.881 ..."
...,...,...,...,...,...,...
2760,190450551044A,19,045,0551,044A,"POLYGON ((2672812.648 1536302.277, 2672794.235..."
2761,1903103572154,19,031,0357,2154,"POLYGON ((2688521.455 1517811.416, 2688514.987..."
2762,190450559051A,19,045,0559,051A,"POLYGON ((2673600.109 1539073.138, 2673613.462..."
2763,1902500030038,19,025,0003,0038,"POLYGON ((2684928.825 1535610.012, 2684933.793..."


In [ ]:
agebTam = gpd.read_file('28a.shp')
agebTam

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry
0,2800100010478,28,001,0001,0478,"POLYGON ((2865811.694 1340928.687, 2865803.906..."
1,2800100010247,28,001,0001,0247,"POLYGON ((2865998.646 1340736.892, 2866011.88 ..."
2,2800100010548,28,001,0001,0548,"POLYGON ((2864982.488 1340468.413, 2865141.097..."
3,2800100010571,28,001,0001,0571,"POLYGON ((2867403.43 1339351.341, 2867402.917 ..."
4,2800100010497,28,001,0001,0497,"POLYGON ((2865211.29 1340703.94, 2865207.546 1..."
...,...,...,...,...,...,...
2435,2804300010389,28,043,0001,0389,"POLYGON ((2811495.919 1221535.842, 2811487.723..."
2436,2804300010162,28,043,0001,0162,"POLYGON ((2813269.725 1222353.944, 2813214.866..."
2437,2804300010181,28,043,0001,0181,"POLYGON ((2812436.843 1222082.102, 2812456.981..."
2438,280430001029A,28,043,0001,029A,"POLYGON ((2811075.575 1221999.809, 2811073.013..."


In [ ]:
agebr = gpd.read_file('19ar.shp')
agebr['CVE_LOC'] = '0001'

In [ ]:
agebrTam = gpd.read_file('28ar.shp')
agebrTam['CVE_LOC'] = '0001'

In [ ]:
# Unir shapefiles
ageb = ageb.to_crs(epsg=4326)
agebr = agebr.to_crs(epsg=4326)
agebTam = agebTam.to_crs(epsg=4326)
agebrTam = agebrTam.to_crs(epsg=4326)

ageb_all = pd.concat([
    ageb,
    agebr,
    agebTam,
    agebrTam
]).reset_index(drop=True)

In [ ]:
ageb_all

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry
0,1900100010021,19,001,0001,0021,"POLYGON ((-100.39902 25.95023, -100.39815 25.9..."
1,1900100010036,19,001,0001,0036,"POLYGON ((-100.39382 25.94482, -100.39363 25.9..."
2,1900100010055,19,001,0001,0055,"POLYGON ((-100.39116 25.93867, -100.39116 25.9..."
3,1900100010040,19,001,0001,0040,"POLYGON ((-100.39218 25.94863, -100.38411 25.9..."
4,1900200010230,19,002,0001,0230,"POLYGON ((-99.53602 26.31221, -99.53612 26.311..."
...,...,...,...,...,...,...
6557,280430088,28,043,0001,0088,"POLYGON ((-98.97019 22.96148, -98.9705 22.9595..."
6558,28043011A,28,043,0001,011A,"POLYGON ((-98.8881 22.88688, -98.88907 22.8805..."
6559,280430209,28,043,0001,0209,"POLYGON ((-98.96416 23.15164, -98.95819 23.150..."
6560,280430213,28,043,0001,0213,"POLYGON ((-99.02061 23.13896, -99.01575 23.139..."


In [ ]:
from shapely.geometry import Point

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# 1. Crear geometría desde lat/lon
df_oxxo['geometry'] = df_oxxo.apply(lambda row: Point(row['LONGITUD_NUM'], row['LATITUD_NUM']), axis=1)

# 2. Convertir en GeoDataFrame con CRS geográfico
gdf = gpd.GeoDataFrame(df_oxxo, geometry='geometry', crs='EPSG:4326')

# 3. Reproyectar al CRS del shapefile
gdf = gdf.to_crs(ageb_all.crs)

# 4. Join espacial para obtener AGEBS
gdf_joined = gpd.sjoin(
    gdf,
    ageb_all,
    how='left',
    predicate='within'
)

''''# Asegúrate que los códigos son string con ceros a la izquierda (formato oficial)
gdf_joined['CVE_MUN'] = gdf_joined['CVE_MUN'].astype(str).str.zfill(2)
gdf_joined['CVE_LOC'] = gdf_joined['CVE_LOC'].astype(str).str.zfill(4)
gdf_joined['CVE_AGEB'] = gdf_joined['CVE_AGEB'].astype(str).str.zfill(4)'''

# Crear columna llave
gdf_joined['LLAVE_AGEB'] = (
    gdf_joined['CVE_ENT'] +
    gdf_joined['CVE_MUN'] +
    gdf_joined['CVE_LOC'] +
    gdf_joined['CVE_AGEB']
)

# 5. Resultado: columna CVE_AGEB agregada
df_con_ageb = pd.DataFrame(gdf_joined.drop(columns='geometry'))  # si quieres como df normal

In [ ]:
df_con_ageb

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,...,Farmacias,Usados,PORCENTAJE_CUMPLIMIENTO,index_right,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,LLAVE_AGEB
0,680,1,C,Hogar,102.360000,13.0,7.868938,25.65488,-100.21207,Hogar Reunión,...,0,0,1.327246,2389,1902600012046,19,026,0001,2046,1902600012046
1,730,1,C,Hogar,97.430000,14.0,7.193831,25.66315,-100.22738,Hogar Reunión,...,0,0,2.736564,2340,1902600011993,19,026,0001,1993,1902600011993
2,650,1,C,Hogar,117.010000,13.0,8.705447,25.66404,-100.22993,Hogar Reunión,...,0,0,1.964698,2281,1902600011989,19,026,0001,1989,1902600011989
3,670,1,C,Base,109.760000,13.0,8.291475,25.66508,-100.26338,Hogar Reunión,...,0,0,1.646274,2306,1902600011438,19,026,0001,1438,1902600011438
4,800,1,C,Peatonal,120.480242,14.0,8.587192,25.69367,-100.21433,Parada Técnica,...,0,0,1.244958,2295,1902600010232,19,026,0001,0232,1902600010232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,100.258370,10.0,8.952197,25.60253,-99.99317,Hogar Reunión,...,0,0,1.414713,395,1900900010250,19,009,0001,0250,1900900010250
101,130,6,CD,Base,110.060000,10.0,9.489423,25.36439,-100.10202,Hogar Reunión,...,0,0,4.021708,1705,1904900010203,19,049,0001,0203,1904900010203
102,320,6,C,Hogar,122.000000,10.0,10.171192,25.57214,-100.01371,Hogar Reunión,...,0,0,2.380020,418,1900900010655,19,009,0001,0655,1900900010655
103,510,6,B,Hogar,110.952073,13.0,8.359542,25.19747,-99.83198,Hogar Reunión,...,0,0,1.919734,2839,190380055,19,038,0001,0055,1903800010055


In [ ]:
df_con_ageb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105 entries, 0 to 104
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TIENDA_ID                   105 non-null    int64  
 1   PLAZA_CVE                   105 non-null    int64  
 2   NIVELSOCIOECONOMICO_DES     105 non-null    object 
 3   ENTORNO_DES                 105 non-null    object 
 4   MTS2VENTAS_NUM              105 non-null    float64
 5   PUERTASREFRIG_NUM           105 non-null    float64
 6   CAJONESESTACIONAMIENTO_NUM  105 non-null    float64
 7   LATITUD_NUM                 105 non-null    float64
 8   LONGITUD_NUM                105 non-null    float64
 9   SEGMENTO_MAESTRO_DESC       105 non-null    object 
 10  LID_UBICACION_TIENDA        105 non-null    object 
 11  Seven-Eleven                105 non-null    int64  
 12  Abarrotes                   105 non-null    int64  
 13  Otros                       105 non-null

In [ ]:
## merge de los datos
df_merged = df_con_ageb.merge(
    demograf_all,
    left_on='LLAVE_AGEB',
    right_on='llaveEnt',
    how='left'
)

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TIENDA_ID                   105 non-null    int64  
 1   PLAZA_CVE                   105 non-null    int64  
 2   NIVELSOCIOECONOMICO_DES     105 non-null    object 
 3   ENTORNO_DES                 105 non-null    object 
 4   MTS2VENTAS_NUM              105 non-null    float64
 5   PUERTASREFRIG_NUM           105 non-null    float64
 6   CAJONESESTACIONAMIENTO_NUM  105 non-null    float64
 7   LATITUD_NUM                 105 non-null    float64
 8   LONGITUD_NUM                105 non-null    float64
 9   SEGMENTO_MAESTRO_DESC       105 non-null    object 
 10  LID_UBICACION_TIENDA        105 non-null    object 
 11  Seven-Eleven                105 non-null    int64  
 12  Abarrotes                   105 non-null    int64  
 13  Otros                       105 non

In [ ]:
print(f"Total con datos demográficos agregados: {df_merged['Llave'].notna().sum()} de {len(df_merged)}")

Total con datos demográficos agregados: 100 de 105


In [ ]:
df_merged.fillna(0, inplace=True)

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TIENDA_ID                   105 non-null    int64  
 1   PLAZA_CVE                   105 non-null    int64  
 2   NIVELSOCIOECONOMICO_DES     105 non-null    object 
 3   ENTORNO_DES                 105 non-null    object 
 4   MTS2VENTAS_NUM              105 non-null    float64
 5   PUERTASREFRIG_NUM           105 non-null    float64
 6   CAJONESESTACIONAMIENTO_NUM  105 non-null    float64
 7   LATITUD_NUM                 105 non-null    float64
 8   LONGITUD_NUM                105 non-null    float64
 9   SEGMENTO_MAESTRO_DESC       105 non-null    object 
 10  LID_UBICACION_TIENDA        105 non-null    object 
 11  Seven-Eleven                105 non-null    int64  
 12  Abarrotes                   105 non-null    int64  
 13  Otros                       105 non

In [ ]:
df_merged.drop(columns=['index_right','CVEGEO','CVE_ENT','CVE_LOC','CVE_AGEB','llaveEnt','Llave'])

,TIENDA_ID,PLAZA_CVE,NIVELSOCIOECONOMICO_DES,ENTORNO_DES,MTS2VENTAS_NUM,PUERTASREFRIG_NUM,CAJONESESTACIONAMIENTO_NUM,LATITUD_NUM,LONGITUD_NUM,SEGMENTO_MAESTRO_DESC,...,PROM_HNV,VIVPAR_UT,PSINDER,VIVTOT,TOTHOG,PDER_SS,GRAPROES,POBTOT,P_60YMAS,P_18YMAS
0,680,1,C,Hogar,102.360000,13.0,7.868938,25.65488,-100.21207,Hogar Reunión,...,91.70,29.0,2562.0,3166.0,2830.0,7638.0,10.100714,10232.0,1288.0,7618.0
1,730,1,C,Hogar,97.430000,14.0,7.193831,25.66315,-100.22738,Hogar Reunión,...,55.03,28.0,1293.0,2128.0,1980.0,5754.0,11.888857,7056.0,1042.0,5538.0
2,650,1,C,Hogar,117.010000,13.0,8.705447,25.66404,-100.22993,Hogar Reunión,...,64.26,16.0,1360.0,2074.0,1870.0,5312.0,11.614062,6672.0,1306.0,5306.0
3,670,1,C,Base,109.760000,13.0,8.291475,25.66508,-100.26338,Hogar Reunión,...,53.26,15.0,530.0,1146.0,1036.0,3004.0,7.6945,3542.0,918.0,2936.0
4,800,1,C,Peatonal,120.480242,14.0,8.587192,25.69367,-100.21433,Parada Técnica,...,67.43,26.0,875.0,1774.0,1576.0,4160.0,9.234878,5042.0,1482.0,4116.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,720,6,C,Hogar,100.258370,10.0,8.952197,25.60253,-99.99317,Hogar Reunión,...,82.38,58.0,748.0,1004.0,856.0,2196.0,8.695526,2948.0,504.0,2146.0
101,130,6,CD,Base,110.060000,10.0,9.489423,25.36439,-100.10202,Hogar Reunión,...,97.31,215.0,690.0,2558.0,2047.0,5908.0,9.43,6662.0,1034.0,4802.0
102,320,6,C,Hogar,122.000000,10.0,10.171192,25.57214,-100.01371,Hogar Reunión,...,52.51,10.0,750.0,2182.0,2008.0,5872.0,9.619189,6664.0,670.0,4974.0
103,510,6,B,Hogar,110.952073,13.0,8.359542,25.19747,-99.83198,Hogar Reunión,...,0.00,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [ ]:
df_merged.to_csv('df_demograficas_test.csv',index=False)